In [1]:
import numpy as np
import pandas as pd
import sklearn


In [2]:
data=pd.read_csv('/home/harshita/Documents/SPE/Original Loan Approver/SmartLoadApproverAsServiceUsingMachineLearning/data/loan.csv',engine='python')

In [3]:
data.head()

,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc,bad_loan,longest_credit_length,verification_status
0,5000,36 months,10.65,10.0,RENT,24000.0,credit_card,AZ,27.65,0.0,83.7,9.0,0,26.0,verified
1,2500,60 months,15.27,0.0,RENT,30000.0,car,GA,1.00,0.0,9.4,4.0,1,12.0,verified
2,2400,36 months,15.96,10.0,RENT,12252.0,small_business,IL,8.72,0.0,98.5,10.0,0,10.0,not verified
3,10000,36 months,13.49,10.0,RENT,49200.0,other,CA,20.00,0.0,21.0,37.0,0,15.0,verified
4,5000,36 months,7.90,3.0,RENT,36000.0,wedding,AZ,11.20,0.0,28.3,12.0,0,7.0,verified


In [4]:
(data.isna()).sum()

loan_amnt                   0
term                        0
int_rate                    0
emp_length               5804
home_ownership              0
annual_inc                  4
purpose                     0
addr_state                  0
dti                         0
delinq_2yrs                29
revol_util                193
total_acc                  29
bad_loan                    0
longest_credit_length      29
verification_status         0
dtype: int64

In [5]:
### Replace missing entries

data['emp_length'].fillna(data['emp_length'].median(),inplace=True)
data['annual_inc'].fillna(data['annual_inc'].median(),inplace=True)
data['delinq_2yrs'].fillna(data['delinq_2yrs'].median(),inplace=True)
data['revol_util'].fillna(data['revol_util'].median(),inplace=True)
data['total_acc'].fillna(data['total_acc'].median(),inplace=True)
data['longest_credit_length'].fillna(data['longest_credit_length'].median(),inplace=True)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163987 entries, 0 to 163986
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   loan_amnt              163987 non-null  int64  
 1   term                   163987 non-null  object 
 2   int_rate               163987 non-null  float64
 3   emp_length             163987 non-null  float64
 4   home_ownership         163987 non-null  object 
 5   annual_inc             163987 non-null  float64
 6   purpose                163987 non-null  object 
 7   addr_state             163987 non-null  object 
 8   dti                    163987 non-null  float64
 9   delinq_2yrs            163987 non-null  float64
 10  revol_util             163987 non-null  float64
 11  total_acc              163987 non-null  float64
 12  bad_loan               163987 non-null  int64  
 13  longest_credit_length  163987 non-null  float64
 14  verification_status    163987 non-nu

In [7]:
## One hot encoding categorical variables
categorical_variables=['term','home_ownership','purpose','addr_state','verification_status']

Interest_rates=data['int_rate']
Verification_status=data['bad_loan']
#Verification_status=np.where(Verification_status=='verified',0,1)

train_ds=data.drop(columns=['int_rate','bad_loan'],inplace=False)

In [8]:
data['home_ownership'].unique()

array(['RENT', 'OWN', 'MORTGAGE', 'OTHER', 'NONE', 'ANY'], dtype=object)

In [9]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer



In [27]:
X_train_rate,X_test_rate,Y_train_rate,Y_test_rate=train_test_split(train_ds,Interest_rates,random_state=42)
X_train_verify,X_test_verify,Y_train_verify,Y_test_verify=train_test_split(train_ds,Verification_status,random_state=42)

In [21]:
col_transformer=ColumnTransformer([('encoder',OneHotEncoder(),categorical_variables)],remainder='passthrough')
#sample=col_transformer.fit_transform(X_train_rate)

In [29]:
X_train_verify['home_ownership'].unique()

array(['MORTGAGE', 'RENT', 'OWN', 'OTHER', 'NONE', 'ANY'], dtype=object)

In [30]:
from sklearn.linear_model import LinearRegression
pipeline_rate=Pipeline([('onehotencoder',col_transformer),('Linear_regressor',LinearRegression())])#

pipeline_rate.fit(X_train_rate,Y_train_rate)
#pipeline_rate.score(X_test_rate,Y_test_rate)
#sample_test=col_transformer.fit_transform(X_test_rate)
#pipeline_rate.score(sample_test,Y_test_rate)

Pipeline(steps=[('onehotencoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encoder', OneHotEncoder(),
                                                  ['term', 'home_ownership',
                                                   'purpose', 'addr_state',
                                                   'verification_status'])])),
                ('Linear_regressor', LinearRegression())])

In [59]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(pipeline_rate.predict(X_test_rate),Y_test_rate)

In [60]:
mse

11.827096700346198

In [33]:
from sklearn.linear_model import LogisticRegression
pipeline_verify=Pipeline([('onehotencoder',col_transformer),('classifier',LogisticRegression())])
pipeline_verify.fit(X_train_verify,Y_train_verify)
pipeline_verify.score(X_test_verify,Y_test_verify)

0.8136936849037735

In [34]:
y_proba = pipeline_verify.predict_proba(X_test_verify)
print(y_proba)

[[0.739429   0.260571  ]
 [0.92640352 0.07359648]
 [0.76985462 0.23014538]
 ...
 [0.95019574 0.04980426]
 [0.75830257 0.24169743]
 [0.89012516 0.10987484]]


In [35]:
from sklearn.metrics import roc_auc_score
roc_score=roc_auc_score(Y_test_verify,pipeline_verify.predict(X_test_verify))
print(roc_score)

0.5


In [85]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [86]:
class BankLoanVerifier:
    def __init__(self,X,Y):
        super(BankLoanVerifier).__init__()
        self.X=X
        self.Y=Y
        
    def preprocess(self):
        self.X['emp_length'].fillna(self.X['emp_length'].median(),inplace=True)
        self.X['annual_inc'].fillna(self.X['annual_inc'].median(),inplace=True)
        self.X['delinq_2yrs'].fillna(self.X['delinq_2yrs'].median(),inplace=True)
        self.X['revol_util'].fillna(self.X['revol_util'].median(),inplace=True)
        self.X['total_acc'].fillna(self.X['total_acc'].median(),inplace=True)
        self.X['longest_credit_length'].fillna(self.X['longest_credit_length'].median(),inplace=True)
        
        #self.Y=np.where(self.Y=='verified',0,1)
        
    def split_dataset(self):
        self.X_train_verify,self.X_test_verify,self.Y_train_verify,self.Y_test_verify=train_test_split(self.X,self.Y,random_state=42)
        
    def train(self):
        self.preprocess()
        self.split_dataset()
        categorical_variables=['term','home_ownership','purpose','addr_state','verification_status']
        col_transformer=ColumnTransformer([('encoder',OneHotEncoder(),categorical_variables)],remainder='passthrough')
        self.pipeline_verify=Pipeline([('onehotencoder',col_transformer),('scaler',StandardScaler(with_mean=False)),('classifier',SVC())]) #LogisticRegression(solver='liblinear')
        self.pipeline_verify.fit(self.X_train_verify,self.Y_train_verify)
        return self.X_train_verify,self.X_test_verify,self.Y_train_verify,self.Y_test_verify
    
    def predict(self,X_test):
        Y_predicted=self.pipeline_verify.predict(X_test)
        return Y_predicted
    
    def roc_auc_score(self,X_test,Y_test):
        Y_test_pred=self.pipeline_verify.predict(X_test)
        roc_score=roc_auc_score(Y_test,Y_test_pred)
        return roc_auc_score
    
    def predict_proba(self,X_test,class_name):
        y_proba = self.pipeline_verify.predict_proba(X_test)[0][class_name]
        return y_proba

In [ ]:
data_new=pd.read_csv('/home/harshita/Documents/SPE/Original Loan Approver/SmartLoadApproverAsServiceUsingMachineLearning/data/loan.csv',engine='python')
Y_loan=data_new['bad_loan']
Y_rate=data_new['int_rate']
data_new.drop(columns=['bad_loan','int_rate'],inplace=True)
verifier=BankLoanVerifier(data_new,Y_loan)
X_train_verify,X_test_verify,Y_train_verify,Y_test_verify=verifier.train()
Y_pred=verifier.predict(X_test_verify)
print(roc_auc_score(Y_test_verify,Y_pred))

In [39]:
X_test_verify.iloc[-1]

loan_amnt                              9750
term                              36 months
emp_length                                4
home_ownership                         RENT
annual_inc                            21600
purpose                  debt_consolidation
addr_state                               CO
dti                                    8.56
delinq_2yrs                               1
revol_util                                0
total_acc                                34
longest_credit_length                    10
verification_status                verified
Name: 2150, dtype: object

In [40]:
one_sample=pd.DataFrame(dict({'loan_amnt':[5000],'term':['60 months'],'emp_length':[12],'home_ownership':['RENT'],
                             'annual_inc':[120000],'purpose':['credit_card'],'addr_state':['MN'],'dti':[9.08],
                             'delinq_2yrs':[1],'revol_util':[50],'total_acc':[60],'longest_credit_length':[20],'verification_status':['verified']}))

In [41]:
one_sample

,loan_amnt,term,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc,longest_credit_length,verification_status
0,5000,60 months,12,RENT,120000,credit_card,MN,9.08,1,50,60,20,verified


In [42]:
verifier.predict(one_sample)[0]

0

In [65]:
import pickle
def pickle_model(model,path='./BadLoanVerifier.pkl'):
        with open(path, 'wb') as f:
            pickle.dump(model, f)
            print("Pickled model at {}".format(path))
            
def unpickle_model(path='./BadLoanVerifier.pkl'):
    with open(path,'rb') as f:
        model=pickle.load(f)
        return model

In [56]:
class InterestRatePredictor:
    def __init__(self,X,Y):
        super(InterestRatePredictor).__init__()
        self.X=X
        self.Y=Y
        
    def preprocess(self):
        self.X['emp_length'].fillna(self.X['emp_length'].median(),inplace=True)
        self.X['annual_inc'].fillna(self.X['annual_inc'].median(),inplace=True)
        self.X['delinq_2yrs'].fillna(self.X['delinq_2yrs'].median(),inplace=True)
        self.X['revol_util'].fillna(self.X['revol_util'].median(),inplace=True)
        self.X['total_acc'].fillna(self.X['total_acc'].median(),inplace=True)
        self.X['longest_credit_length'].fillna(self.X['longest_credit_length'].median(),inplace=True)
        
    def split_dataset(self):
        self.X_train_rate,self.X_test_rate,self.Y_train_rate,self.Y_test_rate=train_test_split(self.X,self.Y,random_state=42)
        
    def train(self):
        self.preprocess()
        self.split_dataset()
        categorical_variables=['term','home_ownership','purpose','addr_state','verification_status']
        col_transformer=ColumnTransformer([('encoder',OneHotEncoder(),categorical_variables)],remainder='passthrough')
        self.pipeline_rate=Pipeline([('onehotencoder',col_transformer),('rate_predictor',LinearRegression())])
        self.pipeline_rate.fit(self.X_train_rate,self.Y_train_rate)
        return self.X_train_rate,self.X_test_rate,self.Y_train_rate,self.Y_test_rate
    
    def predict(self,X_test):
        Y_predicted=self.pipeline_rate.predict(X_test)
        return Y_predicted
    
    def calc_mse_score(self,X_test,Y_test):
        Y_test_pred=self.pipeline_rate.predict(X_test)
        mse_score=mean_squared_error(Y_test,Y_test_pred)
        return mse_score
   

In [66]:

predictor=InterestRatePredictor(data_new,Y_rate)
X_train_rate,X_test_rate,Y_train_rate,Y_test_rate=predictor.train()
Y_pred=predictor.predict(X_test_rate)
print(predictor.calc_mse_score(X_test_rate,Y_test_rate))
reg=pickle_model(predictor,'./InterestRatePredictor.pkl')

11.827096700346198
Pickled model at ./InterestRatePredictor.pkl
